In [ ]:
"""
Produce LandScape File
"""

con_db = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'fireloc_db'
}

db_schema = {
    "GRID" : 'grid_ref', "GRID_PK" : 'gid', 'CELLID' : 'cellid',
    "DATA" : 'datasets', 'DATA_PK' : 'did', 'SLUG' : 'slug',
    "REL"  : 'rel_grid_data', 'GRID_FK' : 'gid', "DATA_FK" : 'did',
    "SHP"  : 'shp', 'RST' : 'rst'
}

sdi_url = 'https://vgi.mat.uc.pt/api/sdi/down-dataset/'
#sdi_url = 'http://127.0.0.1:8181/api/sdi/down-dataset/'
auth_dt = ('kurosaki', 'zangetsu')

lmt = '/home/jasp/fireloc/lmt_coimbra.shp'
epsg = 3763

cos_slug = 'cos15_l3'
dem_slug = 'dem888_50'
tcd_slug = 'tcd15'

climatic_data = '/home/jasp/fireloc/data_meteo.xlsx'
climatic_sheet = 'coimbra_wsx'
climatic_elevation = 171

procedure = '/home/jasp/fireloc/coimbra_data'

import os; import codecs
import json
from osgeo import gdal
import numpy as np
from glass.gt.prop.ext import get_ext
from glass.g.to import shpext_to_boundary
from glass.sql.fm import q_to_obj
from glass.gt.nop.mos import rsts_to_mosaic
from glass.gt.nop.rcls import rcls_rst
from glass.gt.wenv.grs import run_grass
from glass.fm import tbl_to_obj
from glass.gt.nop.alg import floatrst_to_intrst
from glass.g.prop.img import get_cell_size, get_nd
from glass.gt.torst import obj_to_rst
from glass.to.web import get_file_post
from glass.pyt.oss import del_file
from glass.gt.nop.rmp import resample_by_majority

datasets = {'LULC' : cos_slug, 'DEM' : dem_slug, 'TCD' : tcd_slug}

In [ ]:
"""
Generate file with climatic data
"""

climatic_wxs = os.path.join(procedure, 'climatic_txt.wxs')

if os.path.exists(climatic_wxs):
    del_file(climatic_wxs)

meteo_df = tbl_to_obj(climatic_data, sheet=climatic_sheet)

txt_data = (
    'RAWS_ELEVATION: {}\n'
    'RAWS_WINDS: Ave\n'
    'RAWS_UNITS: Metric\n\n'
    'Year  Mth  Day   Time    Temp     RH    HrlyPcp   WindSpd  WindDir CloudCov\n{}'
)
__txt = ''

for idx, row in meteo_df.iterrows():
    __txt += (
        '{year}{a}{month}{b}{day}{c}{tim}{d}{temp}{e}{rh}{f}'
        '{prec}.00{g}{wspeed}{h}{wdir}{i}{cloud}\n'
    ).format(
        year=str(row.Year), a=' ' * 3,
        month=str(row.Mth) if len(str(row.Mth)) == 2 else ' ' + str(row.Mth),
        b=' ' * 3,
        day=str(row.Day) if len(str(row.Day)) == 2 else ' ' + str(row.Day),
        c=' ' * 3, tim=str(row.Time), d=' ' * 6,
        temp=str(row.Temp) if len(str(row.Temp)) == 2 else ' ' + str(row.Temp),
        e=' ' * 5, rh=str(row.RH) if len(str(row.RH)) == 2 else ' ' + str(row.RH),
        f=' ' * 4, prec=str(row.HrlyPcp) if len(str(row.HrlyPcp)) == 4 else \
            ' ' * (4 -  len(str(row.HrlyPcp))) + str(row.HrlyPcp),
        g=' ' * 6, wspeed=str(row.WindSpd) if len(str(row.WindSpd)) == 3 else \
            ' ' * (3 - len(str(row.WindSpd))) + str(row.WindSpd),
        h=' ' * 5, wdir=str(row.WindDir) if len(str(row.WindDir)) == 3 else \
            ' ' * (3 - len(str(row.WindDir))) + str(row.WindDir),
        i=' ' * 4, cloud=str(row.CloudCov) if len(str(row.CloudCov)) == 2 else ' ' + str(row.CloudCov)
    )

with codecs.open(climatic_wxs, 'w', encoding='utf-8') as txt:
    txt.write(txt_data.format(str(climatic_elevation), __txt))

In [ ]:
# Get Extent Geometry
lmt_geom = shpext_to_boundary(lmt, out_srs=3763).ExportToWkt()

for d in datasets:
    data = json.dumps({
        'geom' : lmt_geom, 'dataset' : datasets[d]
    })
    
    of = os.path.join(procedure, '{}.tif'.format(datasets[d]))
    
    if os.path.exists(of):
        del_file(of)
    
    status, ff = get_file_post(sdi_url, of, data, auth_dt)
    
    datasets[d] = of

In [ ]:
"""
Resample LULC and TCD Rasters based on them
"""

# Open DEM Raster
dem_src = gdal.Open(datasets["DEM"])

# Get DEM Cellsize
dem_cs = int(get_cell_size(dem_src)[0])

# Check if COS RST has the same cellsize than dem
lulc_src = gdal.Open(datasets["LULC"])
lulc_cs = int(get_cell_size(lulc_src)[0])

if lulc_cs == dem_cs:
    lulc_rst = lulc_rst
elif lulc_cs < dem_cs:
    lulc_rst = resample_by_majority(dem_src, lulc_src, os.path.join(procedure, 'gen_lulc.tif'))
else:
    raise ValueError('script doesn\'t know what to do...')

# Reclass COS
# CLS to Fuel Model
rules = {
    (0, 11) : 90,
    (11, 19) : 1,
    20 : 5,
    21 : 1,
    22 : 9,
    23 : 6,
    24 : 1,
    25 : 6,
    26 : 8,
    27 : 90,
    (27, 32) : 98
}

rcls_lulc = rcls_rst(lulc_rst, rules, os.path.join(procedure, 'fuel_rst.tif'))

In [ ]:
# Get Tree Canocopy cover

# Check if COS RST has the same cellsize than dem
tcd_src = gdal.Open(datasets["TCD"])
tcd_cs = int(get_cell_size(tcd_src)[0])

if tcd_cs == dem_cs:
    tcd_rst = tcd_rst
elif tcd_cs < dem_cs:
    tcd_rst = resample_by_majority(dem_src, tcd_src, os.path.join(procedure, 'gen_tcd.tif'))
else:
    raise ValueError('script doesn\'t know what to do...')

# Reclass
rules = {0 : 99, (0, 20) : 1, (20, 50) : 2, (50, 80) : 3, (80, 100) : 4}

tcc_rst = rcls_rst(tcd_rst, rules, os.path.join(procedure, 'tcc.tif'))

In [ ]:
# Create Slope and Aspect Raster
grsbase = run_grass(procedure, grassBIN='grass78', location='dem_asp', srs=epsg)

import grass.script as grass
import grass.script.setup as gsetup
gsetup.init(grsbase, procedure, 'dem_asp', 'PERMANENT')

from glass.gt.torst import rst_to_grs, grs_to_rst
from glass.gt.wenv.grs import rst_to_region
from glass.gt.nop.surf import slope, aspect

# Configure region
rst_ext = rst_to_grs(datasets["DEM"], 'dem')
rst_to_region(rst_ext)

# Produce slope raster
dclv = slope(rst_ext, 'slope_rst', api='grass')

# Produce aspect raster
expo = aspect(rst_ext, 'aspect_rst', api='grass', from_north=True)

# Export data
dclv = grs_to_rst(dclv, os.path.join(procedure, 'dclv.tif'))
expo = grs_to_rst(expo, os.path.join(procedure, 'aspect.tif'))

"""
Convert Float Rasters to Integers Rasters
"""

for r in [datasets["DEM"], dclv, expo]:
    floatrst_to_intrst(r, os.path.join(procedure, 'int_{}'.format(os.path.basename(r))))